In [1]:
import psycopg2
import numpy as np
import pandas as pd
import pandas.io.sql as sqlio

In [2]:
# Load Skill Builder Information

exp_norm_map = pd.read_csv('experiment_information/exp_norm_map.csv')
normal_skill_builders_in_folder = pd.read_csv('experiment_information/normal_skill_builders_in_folder.csv')
normal_replaced_skill_builders = pd.read_csv('experiment_information/normal_replaced_skill_builders.csv')

In [3]:
# Dictionary of Experiment to Normal Skill Builders
exp_norm_dict = dict(zip(exp_norm_map['experiment_id'], exp_norm_map['normal_id']))

# Array of All Normal Skill Builders
all_norm_skill_builders = np.unique(pd.concat([normal_skill_builders_in_folder['sequence_id'], normal_replaced_skill_builders['sequence_id']]))

In [4]:
# Database Credentials

US = 'postgres'
PA = 'RuePierre0614'
HO = 'dev.tng.cs.wpi.edu'
PO = 5432
DB = 'cas_core'

# List of Experiment Problem Sets
experiment_skill_builders =  "('" + "','".join(exp_norm_dict.keys()) + "')"

# List of Non-Experiment Problem Sets
normal_skill_builders = "('" + "','".join(all_norm_skill_builders) + "')"

In [5]:
# Run the script that creates tables

conn = psycopg2.connect(user=US, password=PA, host=HO, port=PO, database=DB)
conn.set_session(autocommit=True)
with open(f'sql/make_data.sql', 'rt') as fin:
    with open(f'sql/tmp.sql', 'wt') as fout:
        for line in fin:
            fout.write(line.replace('VAR_EXP_SB', experiment_skill_builders)
                           .replace('VAR_NORM_SB', normal_skill_builders))
conn.cursor().execute(open(f'sql/tmp.sql', 'r').read())

In [6]:
sqlio.read_sql_query('select * from remnant_actions', conn).to_csv('raw_data/remnant_actions.csv', index=False)
sqlio.read_sql_query('select * from remnant_inputs', conn).to_csv('raw_data/remnant_inputs.csv', index=False)
sqlio.read_sql_query('select * from remnant_targets', conn).to_csv('raw_data/remnant_targets.csv', index=False)
sqlio.read_sql_query('select * from experiment_actions', conn).to_csv('raw_data/experiment_actions.csv', index=False)
sqlio.read_sql_query('select * from experiment_inputs', conn).to_csv('raw_data/experiment_inputs.csv', index=False)
sqlio.read_sql_query('select * from experiment_targets', conn).to_csv('raw_data/experiment_targets.csv', index=False)

In [7]:
conn.close()